### Loading the libraries

In [137]:
import pandas as pd
import pyodbc
import warnings
warnings.simplefilter("ignore", UserWarning)

import os
from dotenv import load_dotenv
load_dotenv()

from sqlalchemy import create_engine, text
import sqlalchemy


### Setting up the Database connection with MySQL

In [146]:
conn = pyodbc.connect(
    "DRIVER={MySQL ODBC 8.0 Unicode Driver};"
    f"SERVER={os.getenv('server')};"
    f"PORT={os.getenv('port')};"
    f"DATABASE={os.getenv('database')};"
    f"USER={os.getenv('user')};"
    f"PASSWORD={os.getenv('password')};"
    "OPTION=3;"
)

#conn.close()

#### Testing the DB connection

In [148]:
basic_query = "SELECT * FROM retail_sales"
retail_sales = pd.read_sql(basic_query, conn)

In [150]:
len(retail_sales), retail_sales.head(5)

(2000,
    transactions_id   sale_date sale_time  customer_id  gender   age  \
 0                1  2022-12-16  19:10:00           50    Male  34.0   
 1                2  2022-06-24  10:07:00          104  Female  26.0   
 2                3  2022-06-14  07:08:00          114    Male  50.0   
 3                4  2023-08-27  18:12:00            3    Male  37.0   
 4                5  2023-09-05  22:10:00            3    Male  30.0   
 
       category  quantity  price_per_unit   cogs  total_sale  
 0       Beauty       3.0            50.0   16.0       150.0  
 1     Clothing       2.0           500.0  135.0      1000.0  
 2  Electronics       1.0            30.0    8.1        30.0  
 3     Clothing       1.0           500.0  200.0       500.0  
 4       Beauty       2.0            50.0   24.0       100.0  )

In [154]:
### Trying SQLAlchemy
# # Define MySQL Connection String
# db_user = os.getenv("user")
# db_password = os.getenv("password")
# db_host = os.getenv('server')  # Change to your MySQL server if needed
# db_port = os.getenv('port')  # Default MySQL port
# db_name = os.getenv('database')  # Change this to your desired database name

# # Create SQLAlchemy Engine
# engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# # Test connection
# with engine.connect() as conn:
#     print("✅ Connected to MySQL successfully!")

# #Testing connection using pymysql 
# conn = pymysql.connect(db='sql_project_1', user='root', passwd='Mymummy1@', host='localhost', port=3306)
# conn.close()

### Exploratory Data Analysis

#### Finding the number of rows and columns in the SQL dataset

In [165]:
q1 = """ 
SELECT COUNT(*) as total_transactions FROM retail_sales ;
"""

pd.read_sql(q1, conn)

,total_transactions
0,2000


In [166]:
q2 = """ 
SELECT COUNT(*) AS column_count
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'retail_sales'
"""
pd.read_sql(q2, conn)

,column_count
0,11
